In [16]:
import polars as pl
import pandas as pd
from datetime import timedelta
from functools import reduce

In [17]:
s1_minute_data = pd.read_parquet(
    "/home/thomas/repos/simplify_deployment/data/simplify_1_0/s1_minute_data.parquet"
)
s1_minute_data 

,siCumulative
2019-08-31 22:00:00+00:00,-206.520
2019-08-31 22:01:00+00:00,-220.086
2019-08-31 22:02:00+00:00,-224.004
2019-08-31 22:03:00+00:00,-245.507
2019-08-31 22:04:00+00:00,-263.132
...,...
2023-10-31 22:55:00+00:00,-264.003
2023-10-31 22:56:00+00:00,-258.480
2023-10-31 22:57:00+00:00,-246.817
2023-10-31 22:58:00+00:00,-238.509


In [18]:
feature_list = []
for lag in [25,26,27,28]:
    feature = (
        s1_minute_data
        .shift(freq = timedelta(minutes = lag))
    )
    feature = feature.add_prefix(f"lag_{lag}_")
    feature_list.append(feature)


In [19]:
s1_quarter_data = pd.read_parquet(
    "/home/thomas/repos/simplify_deployment/data/simplify_1_0/s1_quarter_data.parquet"
).asfreq("1min").ffill()
s1_quarter_data

,si_cum_15,loaD_ID_MW,xB_ID_MW,dsO_ID_MW
utc_time,,,,
2019-08-31 22:00:00+00:00,-264.364,8056.13,-1463.8,-4561.0
2019-08-31 22:01:00+00:00,-264.364,8056.13,-1463.8,-4561.0
2019-08-31 22:02:00+00:00,-264.364,8056.13,-1463.8,-4561.0
2019-08-31 22:03:00+00:00,-264.364,8056.13,-1463.8,-4561.0
2019-08-31 22:04:00+00:00,-264.364,8056.13,-1463.8,-4561.0
...,...,...,...,...
2024-02-29 22:41:00+00:00,-175.593,9681.25,972.6,-5141.4
2024-02-29 22:42:00+00:00,-175.593,9681.25,972.6,-5141.4
2024-02-29 22:43:00+00:00,-175.593,9681.25,972.6,-5141.4


In [20]:
qh_list = []
for name in ["loaD_ID_MW","dsO_ID_MW","xB_ID_MW"]:
    for lag in [70,55,40,25,10,-5,-20]:
        feature = (
            s1_quarter_data
            .loc[:,[name]]
            .shift(freq = timedelta(minutes = lag))
        )
        feature = feature.add_prefix(f"lag_{lag}_")
        qh_list.append(feature)

In [21]:
X_qh = reduce(
    lambda a,b: pd.merge(a,b,left_index=True, right_index=True, how = "inner"),
    qh_list
)
X_qh.to_parquet("s1_qh_predictors.parquet")

In [24]:
X_qh


,lag_70_loaD_ID_MW,lag_55_loaD_ID_MW,lag_40_loaD_ID_MW,lag_25_loaD_ID_MW,lag_10_loaD_ID_MW,lag_-5_loaD_ID_MW,lag_-20_loaD_ID_MW,lag_70_dsO_ID_MW,lag_55_dsO_ID_MW,lag_40_dsO_ID_MW,...,lag_10_dsO_ID_MW,lag_-5_dsO_ID_MW,lag_-20_dsO_ID_MW,lag_70_xB_ID_MW,lag_55_xB_ID_MW,lag_40_xB_ID_MW,lag_25_xB_ID_MW,lag_10_xB_ID_MW,lag_-5_xB_ID_MW,lag_-20_xB_ID_MW
utc_time,,,,,,,,,,,,,,,,,,,,,
2019-08-31 23:10:00+00:00,8056.13,7749.84,7732.03,7648.81,7512.23,7484.50,7302.07,-4561.0,-4520.4,-4497.5,...,-4340.4,-4305.5,-4276.4,-1463.8,-1463.8,-1463.8,-1463.8,-1237.0,-1237.0,-1237.0
2019-08-31 23:11:00+00:00,8056.13,7749.84,7732.03,7648.81,7512.23,7484.50,7302.07,-4561.0,-4520.4,-4497.5,...,-4340.4,-4305.5,-4276.4,-1463.8,-1463.8,-1463.8,-1463.8,-1237.0,-1237.0,-1237.0
2019-08-31 23:12:00+00:00,8056.13,7749.84,7732.03,7648.81,7512.23,7484.50,7302.07,-4561.0,-4520.4,-4497.5,...,-4340.4,-4305.5,-4276.4,-1463.8,-1463.8,-1463.8,-1463.8,-1237.0,-1237.0,-1237.0
2019-08-31 23:13:00+00:00,8056.13,7749.84,7732.03,7648.81,7512.23,7484.50,7302.07,-4561.0,-4520.4,-4497.5,...,-4340.4,-4305.5,-4276.4,-1463.8,-1463.8,-1463.8,-1463.8,-1237.0,-1237.0,-1237.0
2019-08-31 23:14:00+00:00,8056.13,7749.84,7732.03,7648.81,7512.23,7484.50,7302.07,-4561.0,-4520.4,-4497.5,...,-4340.4,-4305.5,-4276.4,-1463.8,-1463.8,-1463.8,-1463.8,-1237.0,-1237.0,-1237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 22:21:00+00:00,10035.47,10036.06,10018.80,9932.85,9911.71,9778.57,9681.25,-5365.0,-5367.3,-5337.2,...,-5259.9,-5205.8,-5141.4,858.0,977.2,1015.7,983.6,1002.8,1050.4,972.6
2024-02-29 22:22:00+00:00,10035.47,10036.06,10018.80,9932.85,9911.71,9778.57,9681.25,-5365.0,-5367.3,-5337.2,...,-5259.9,-5205.8,-5141.4,858.0,977.2,1015.7,983.6,1002.8,1050.4,972.6
2024-02-29 22:23:00+00:00,10035.47,10036.06,10018.80,9932.85,9911.71,9778.57,9681.25,-5365.0,-5367.3,-5337.2,...,-5259.9,-5205.8,-5141.4,858.0,977.2,1015.7,983.6,1002.8,1050.4,972.6


In [23]:
y_test

,si_cum_15
utc_time,
2024-01-01 00:14:00+00:00,0.000
2024-01-01 00:29:00+00:00,0.000
2024-01-01 00:44:00+00:00,0.000
2024-01-01 00:59:00+00:00,0.000
2024-01-01 01:14:00+00:00,0.000
...,...
2024-02-29 21:59:00+00:00,-102.969
2024-02-29 22:14:00+00:00,-312.698
2024-02-29 22:29:00+00:00,-157.582
